In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
learning_rate = 0.01


In [4]:
train= pd.read_csv("iris_training.csv",names=["sepal_lenght","sepal_width","petal_length","petal_width","class"])
test= pd.read_csv("iris_test.csv",names=["sepal_lenght","sepal_width","petal_length","petal_width","class"])

In [5]:
train_x=train[["sepal_lenght","sepal_width","petal_length","petal_width"]]
test_x=test[["sepal_lenght","sepal_width","petal_length","petal_width"]]
             

In [6]:
train_x=train_x.convert_objects(convert_numeric=True)
test_x=test_x.convert_objects(convert_numeric=True)

test_x.dtypes

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


sepal_lenght    float64
sepal_width     float64
petal_length    float64
petal_width     float64
dtype: object

In [24]:
test_y.describe()

,class
count,30.000000
mean,1.000000
std,0.742781
min,0.000000
25%,0.250000
50%,1.000000
75%,1.750000
max,2.000000


In [7]:
train_y=pd.DataFrame(train['class'])
test_y=pd.DataFrame(test['class'])
train_y=train_y.convert_objects(convert_numeric=True)
test_y=test_y.convert_objects(convert_numeric=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


In [8]:
train_x=train_x.iloc[1:]
test_x=test_x.iloc[1:]
train_y=train_y.iloc[1:]
test_y=test_y.iloc[1:]

In [9]:
nodes_x=train_x.shape[1]
nodes_y=train_y.shape[1]

In [13]:
nodes_y

1

define the neural netwwork

In [10]:
tf.reset_default_graph()
learning_rate=0.01
g=tf.Graph()
with g.as_default():
    x=tf.placeholder(tf.float32,[None,nodes_x],name="x")
    y=tf.placeholder(tf.float32,[None,nodes_y],name="y")

In [11]:
h1=5
h2=10
h3=5

with tf.name_scope('Neural_Nt'):
    FC1=tf.contrib.layers.fully_connected(inputs=x,num_outputs=h1,
                                          activation_fn=tf.nn.relu,scope="Fully_Conn1")
    FC2=tf.contrib.layers.fully_connected(inputs=
                        FC1,num_outputs=h2,activation_fn=
                                        tf.nn.relu,scope="Fully_Conn2")
    FC3=tf.contrib.layers.fully_connected(inputs=
                        FC2,num_outputs=h3,activation_fn=
                                        tf.nn.relu,scope="Fully_Conn3")
    predict_y=tf.contrib.layers.fully_connected(inputs=FC3,num_outputs=nodes_y,activation_fn=tf.nn.softmax,scope="Out")
    

In [12]:
with tf.name_scope('cost'):
    cost=tf.losses.softmax_cross_entropy(onehot_labels=y,logits=predict_y,scope="Cost_Function")

In [13]:
with g.as_default():
    corect_pred=tf.equal(tf.argmax(predict_y,1,name="Argmax_pred"),tf.argmax(y,1,name="Y_Pred"),name="Correct_Pred")
    accuracy=tf.reduce_mean(tf.cast(corect_pred,tf.float32,name="Cast_Corr_Pred"),name="Accuracy")


In [14]:
with g.as_default():
    optimizer=tf.train.AdagradOptimizer(learning_rate,name="Optimizer").minimize(cost)

In [17]:
logs="log_final"

tf.summary.scalar("Loss",cost)
tf.summary.scalar("Accuracy",accuracy)
merged_summary3=tf.summary.merge_all()

sess=tf.InteractiveSession(graph=g)
init=tf.global_variables_initializer()


/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [18]:
with tf.Session(graph=g) as sess:
    sess.run(init)
    writer=tf.summary.FileWriter(logs,graph=tf.get_default_graph())
    
    train_epochs=1000
    for epoch in range(train_epochs):
       # print(train_y)
        _,c,summary=sess.run([optimizer,cost,merged_summary3],feed_dict={x:train_x,y:train_y})
        writer.add_summary(summary,epoch)

In [ ]:
tf.reduce_mean(cost)

to run tensor board type 

1. python -m tensorboard
2.tensorboard --logdir ./log_final